# First results with 1D Gaussian process datasets

In [ ]:
import os
import sys
import itertools

# If we don't need CUDA, do this before importing TF
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
import pandas as pd
import tqdm
import tqdm.notebook
import scipy.stats
import matplotlib.pyplot as plt
import GPy
from IPython.display import HTML, display

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_visible_devices([gpus[1]], 'GPU')

sys.path.append("/nfs/scistore12/chlgrp/vvolhejn/smooth")

In [ ]:
%load_ext autoreload
%aimport smooth.datasets
%aimport smooth.model
%aimport smooth.analysis
%aimport smooth.callbacks
%aimport smooth.measures
%aimport smooth.util
%autoreload 1

In [ ]:
# dataset = smooth.datasets.GaussianProcessDataset(
#     x_min=-1, x_max=1,
#     samples_train=50, samples_test=200,
# #     seed=123,
#     plot=True,
# )

dataset = smooth.datasets.GaussianProcessDataset(
    samples_train=100,
    lengthscale=0.3,
    plot=True,
    seed=123,
)

In [ ]:
_history_cb = smooth.callbacks.WeightsHistoryCallback(min_snapshots=200)
_model = smooth.model.train_shallow(
    dataset,
    learning_rate=0.03,
    init_scale=10.,
    epochs=100000,
    hidden_size=500,
    batch_size=len(dataset.x_train),
    verbose=0,
    callbacks=[
        tf.keras.callbacks.EarlyStopping("loss", min_delta=1e-5, patience=2000),
        smooth.callbacks.Tqdm(verbose=0),
        tf.keras.callbacks.TerminateOnNaN(),
        _history_cb
    ],
    train_val_split=1.0,
    activation="relu",
)
smooth.analysis.plot_shallow(_model, dataset)
_ani = smooth.analysis.plot_shallow(_model, dataset, weights_history=_history_cb.weights_history)
display(HTML(_ani.to_html5_video()))

In [ ]:
plt.scatter(dataset.x_train, dataset.y_train, color="g")
x = np.linspace(-1, 1, 100)
plt.plot(x, _model.predict(x))

In [ ]:
ow = _model.get_weights()
_model.set_weights(_history_cb.weights_history[max(_history_cb.weights_history.keys())])
smooth.analysis.plot_shallow(_model, dataset)

_model.set_weights(ow)
smooth.analysis.plot_shallow(_model, dataset)

In [ ]:
def train_models(activation, n=2, plot=False):
    init_scales = np.logspace(np.log10(0.1), np.log10(10.), n)
    models = dict()

    for init_scale in init_scales:
        history_cb = smooth.callbacks.WeightsHistoryCallback(min_snapshots=75)

        models[init_scale] = smooth.model.train_shallow(
            dataset,
            learning_rate=0.01 / init_scale,
            init_scale=init_scale,
            epochs=30000,
            hidden_size=400,
            batch_size=len(dataset.x_train),
            verbose=0,
            callbacks=[
                tf.keras.callbacks.EarlyStopping("loss", min_delta=1e-5, patience=2000),
                smooth.callbacks.Tqdm(verbose=0),
                tf.keras.callbacks.TerminateOnNaN(),
                history_cb
            ],
            train_val_split=1.0,
            activation=activation,
        )

        if plot:
            smooth.analysis.plot_shallow(models[init_scale], dataset)
            ani = smooth.analysis.plot_shallow(
                models[init_scale],
                dataset,
                weights_history=history_cb.weights_history,
            )
            display(HTML(ani.to_html5_video()))
        
    return models

In [ ]:
models_relu = train_models("relu", plot=True)

In [ ]:
models_tanh = train_models("tanh", plot=True)

In [ ]:
def plot_measures(models):
    measures = dict()

    for init_scale, model in tqdm.notebook.tqdm(models.items()):
        measures[init_scale] = smooth.measures.get_measures(model, x_test, y_test)

    measure_names = measures[init_scales[0]].keys()

    x = sorted(measures.keys())
    yd = dict(zip(measure_names,[[] for _ in range(len(measure_names))]))

    for init_scale in x:
        for k, v in measures[init_scale].items():
            yd[k].append(v)

    for measure_name in measure_names:
        plt.plot(x, yd[measure_name])
        plt.title(measure_name)
        plt.xscale("log")
        plt.show()
    
    return measures

In [ ]:
plot_measures(models_relu);

In [ ]:
plot_measures(models_tanh);

In [ ]:
os.chdir("/nfs/scistore12/chlgrp/vvolhejn/smooth/logs_debug/0224-115640/")
df = pd.read_feather("measures.feather")
smooth.analysis.remove_constant_columns(df, verbose=True)
df = smooth.analysis.expand_dataset_columns(df)
df["log_dir"] = df["log_dir"].str.split("/").str.get(-1)
df.head()

In [ ]:
def get_interpolation_measures(dataset_names, use_test_set=False):
    res = []
    for dataset_name in tqdm.notebook.tqdm(dataset_names):
        dataset = smooth.datasets.GaussianProcessDataset.from_name(dataset_name)
        model = smooth.model.interpolate_relu_network(dataset, use_test_set)
        measures = smooth.measures.get_measures(
            model,
            dataset.x_test, dataset.y_test,
            include_training_measures=False,
        )
        res.append(measures)
    
    return pd.DataFrame(res, index=dataset_names)

im_train = get_interpolation_measures(df["dataset"].unique())
im_test = get_interpolation_measures(df["dataset"].unique(), use_test_set=True)

In [ ]:
im_delta = im_train - im_test
im_delta.sort_values("seg_total_variation_derivative")

In [ ]:
dataset = smooth.datasets.GaussianProcessDataset.from_name("gp-1-0.1-10")
plt.plot(dataset.x_train, dataset.y_train)
plt.plot(dataset.x_test, dataset.y_test)

In [ ]:
df1 = df[(df["dataset_seed"] == 1) & (df["dataset_lengthscale"] == 0.1) & (df["train_loss"] < 10)]
# df1 = df1[(df1[""])]
print(len(df1))
plt.scatter(df1["dataset_samples_train"], df1["seg_total_variation"], alpha=0.3)
df1 = df1[df1["dataset_samples_train"] == 100]

In [ ]:
dataset = smooth.datasets.GaussianProcessDataset.from_name(df1.iloc[0]["dataset"])
x = dataset.x_test

ax = plt.subplot()
ax.plot(x, dataset.y_test, color="C0")

for i, row in list(df1.iterrows()):
    log_dir = row["log_dir"]
    model = tf.keras.models.load_model(os.path.join(log_dir, "model.h5"))
    y = model.predict(x)
    color = {
        10: "C1",
        30: "C2",
        100: "C3",
    }[row["dataset_samples_train"]]
    ax.plot(x, y, alpha=0.3, color=color)


#     smooth.analysis.plot_shallow(model, dataset, title=log_dir)

In [ ]:
df1

In [ ]:
# %matplotlib notebook
ax = plt.subplot()
ax.plot(df1["train_loss"], df1["test_loss"], marker='o', linestyle="None")
# ax.set_xlim(auto=True)
# df1["train_loss"].max()

In [ ]:
plt.hist(df1["train_loss"], bins=20)

In [ ]:
for col in df.columns:
    if df[col].dtype == "object":
        continue
    plt.hist(df[col], bins=20)
    plt.title(col)
    plt.show()

In [ ]:
df1 = df[df["dataset"].str.match("gp-.*-0.1-30$")]

for i, row in df1.sort_values("train_loss").iterrows():
    log_dir = row["log_dir"]
    model = tf.keras.models.load_model(os.path.join(log_dir, "model.h5"))
    dataset = smooth.datasets.GaussianProcessDataset.from_name(row["dataset"])
    smooth.analysis.plot_shallow(model, dataset, title=log_dir)

In [ ]:
# dataset = smooth.datasets.Dataset(x_train=[-1, 0, 1, 2], y_train =[-1, 1, 0, 1], x_test=[], y_test=[])
dataset = smooth.datasets.GaussianProcessDataset.from_name(df1.iloc[0]["dataset"])
plt.plot(dataset.x_train, dataset.y_train)
model = smooth.model.interpolate_relu_network(dataset)
smooth.analysis.plot_shallow(model, dataset)